In [ ]:
# ============================================================
# 04e_encoding_baselines_synthetic_eval.ipynb
# Baseline encoding comparison:
#   - onehot
#   - target encoding
#   - tfidf (TF-IDF → SVD 64 dims)
#   - entity embeddings (PyTorch)
#
# Compare CTGAN/TVAE synthetic quality the SAME way as 4D.
# Uses same epochs (30), same max synth rows (15k).
# numeric_only is NOT rerun here.
#
# Results stored in:
#   outputs/synthetic_eval_4e/
#
# Combined summary appended to:
#   outputs/synthetic_eval_4d/summary_all_datasets_views.csv
# ============================================================

!pip -q install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.5.2 lightgbm==4.5.0 \
               sdv==1.16.0 ctgan==0.10.2 category_encoders==2.6.3 \
               torch==2.3.1

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, time, re
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sdv.single_table import CTGANSynthesizer, TVAESynthesizer
from sdv.metadata import SingleTableMetadata

import category_encoders as ce
import torch
import torch.nn as nn
import torch.optim as optim

# -------------------------
# Paths / constants
# -------------------------

RSEED = 42
np.random.seed(RSEED)
torch.manual_seed(RSEED)

PROJ    = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
OUT_ROOT = f"{PROJ}/outputs/synthetic_eval_4e"
os.makedirs(OUT_ROOT, exist_ok=True)

print("Project dir:", PROJ)
print("Output dir :", OUT_ROOT)

# ============================================================
# Dataset config (same target-candidate logic as 4D)
# ============================================================

DATASETS = {
    "adult": {
        "path": f"{DATA_DIR}/Adult_clean.csv",
        "target_candidates": ["income_bin", "Income_bin", "income", "Income", "target", "class"]
    },
    "petfinder": {
        "path": f"{DATA_DIR}/Petfinder_clean.csv",
        "target_candidates": ["AdoptionSpeed_bin", "adoption_bin", "AdoptionSpeed"]
    },
    "breast": {
        "path": f"{DATA_DIR}/Breast_clean.csv",
        "target_candidates": ["OS5yr_bin", "os5yr_bin", "OS5yr", "target"]
    }
}

# Same 3 high-card categorical candidates (as 03a)
CANDIDATE_CATS = {
    "adult":      ["occupation", "workclass", "native_country"],
    "petfinder":  ["Breed1", "Color1", "MaturitySize"],
    "breast":     ["TNM_PATH_T", "TNM_PATH_N", "hospid"],   # confirmed correct
}

# Views for 4E
FEATURE_VIEWS = ["onehot", "target", "tfidf", "entity"]

# -------------------------
# Synthetic + model config
# -------------------------

CONFIG = {
    "train_frac": 0.5,

    "ctgan": {
        "epochs": 30,
        "batch_size": 256,
        "pac": 1,
        "verbose": True
    },

    "tvae": {
        "epochs": 30,
        "batch_size": 256,
        "compress_dims": [128, 64]
    },

    "models": {
        "LR": {
            "type": "lr",
            "params": {"max_iter": 1000, "n_jobs": -1}
        },
        "LinSVM": {
            "type": "linsvm",
            "params": {"C": 1.0}
        },
        "RF": {
            "type": "rf",
            "params": {
                "n_estimators": 400,
                "max_depth": None,
                "n_jobs": -1,
                "random_state": RSEED
            }
        },
        "LGBM": {
            "type": "lgbm",
            "params": {
                "n_estimators": 500,
                "num_leaves": 63,
                "random_state": RSEED
            }
        }
    }
}

print("Config loaded.")


Mounted at /content/drive
Project dir: /content/drive/MyDrive/dissertation
Output dir : /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e
Config loaded.


In [ ]:
# ============================================================
# 4E: encoding views (onehot, target, tfidf, entity) + dispatcher
# ============================================================
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from torch.utils.data import TensorDataset, DataLoader


import torch
import torch.nn as nn
import torch.optim as optim

# Safety: max total columns allowed for one-hot view
MAX_OHE_COLS_4E = 200  # you can bump to 256 if Colab handles it fine


# -------------------------
# shared helpers
# -------------------------
def _split_numeric_and_cats(ds_name, X_tr_raw, X_va_raw, X_te_raw):
    """Return (candidate_cats, base_numeric_cols) for a dataset."""
    cats = CANDIDATE_CATS[ds_name]
    base_num_cols = X_tr_raw.select_dtypes(include=["number", "bool"]).columns.tolist()
    base_num_cols = [c for c in base_num_cols if c not in cats]
    return cats, base_num_cols

def _get_base_numeric_cols(df, cats):
    """
    Backwards-compatible helper in case any old cell still uses it.
    Returns numeric/bool columns excluding the candidate categorical ones.
    """
    num_cols = df.select_dtypes(include=["number", "bool"]).columns.tolist()
    return [c for c in num_cols if c not in cats]

# -------------------------
# ONE-HOT VIEW
# -------------------------
def build_onehot_view(ds_name, X_tr_raw, X_va_raw, X_te_raw):
    """
    One-hot encode the candidate categorical variables, and keep all
    non-candidate numeric/bool variables as-is.
    """
    cats, base_num_cols = _split_numeric_and_cats(ds_name, X_tr_raw, X_va_raw, X_te_raw)

    enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

    def _cat_frame(df):
        # Only keep candidate cats that are present in this split
        use_cols = [c for c in cats if c in df.columns]
        if not use_cols:
            # no cats; just return empty frame
            return pd.DataFrame(index=df.index)
        tmp = df[use_cols].astype(str).fillna("Unknown")
        return tmp

    # Fit encoder on TRAIN cats
    X_tr_cat = _cat_frame(X_tr_raw)
    enc.fit(X_tr_cat)

    cat_cols = enc.get_feature_names_out(X_tr_cat.columns).tolist()

    # --- safety check: too wide? ---
    total_cols = len(base_num_cols) + len(cat_cols)
    if total_cols > MAX_OHE_COLS_4E:
        raise ValueError(
            f"One-hot view too wide for {ds_name}: "
            f"{total_cols} columns (limit={MAX_OHE_COLS_4E})."
        )

    def _encode_split(df_raw):
        X_num = df_raw[base_num_cols].copy()
        X_cat = _cat_frame(df_raw)

        if X_cat.shape[1] == 0:
            out = X_num
        else:
            mat = enc.transform(X_cat)
            X_cat_df = pd.DataFrame(mat, columns=cat_cols, index=df_raw.index)
            out = pd.concat([X_num, X_cat_df], axis=1)

        return out.astype(float)

    X_tr_view = _encode_split(X_tr_raw)
    X_va_view = _encode_split(X_va_raw)
    X_te_view = _encode_split(X_te_raw)

    print("[onehot] Encoded shapes (train / val / test):",
          X_tr_view.shape, X_va_view.shape, X_te_view.shape)

    feature_cols = list(X_tr_view.columns)
    return X_tr_view, X_va_view, X_te_view, feature_cols



# -------------------------
# TARGET ENCODING VIEW
# -------------------------
def build_target_view(ds_name, X_tr, X_va, X_te, y_tr):
    """
    Simple target encoding:
      - For each candidate categorical column: E[y | category] on TRAIN.
      - Validation/test: map with train means, unseen → global mean.
      - All non-candidate numeric/bool columns are kept as-is.
    """
    cats, base_num_cols = _split_numeric_and_cats(ds_name, X_tr, X_va, X_te)

    df_tr = X_tr.copy()
    df_tr["__y__"] = y_tr.values
    global_mean = float(df_tr["__y__"].mean())

    # Per-column mapping: category → mean(__y__)
    mapping = {}
    for col in cats:
        if col not in df_tr.columns:
            continue
        s = df_tr[col].astype(str).fillna("Unknown")
        tmp = pd.DataFrame({"cat": s, "__y__": df_tr["__y__"]})
        m = tmp.groupby("cat")["__y__"].mean()
        mapping[col] = m

    def _encode_split(df_split):
        out = df_split[base_num_cols].copy()
        for col in cats:
            if col not in df_split.columns:
                continue
            s = df_split[col].astype(str).fillna("Unknown")
            m = mapping.get(col)
            if m is None:
                out[f"{col}_te"] = global_mean
            else:
                out[f"{col}_te"] = s.map(m).fillna(global_mean)
        return out.astype(float)

    X_tr_view = _encode_split(X_tr)
    X_va_view = _encode_split(X_va)
    X_te_view = _encode_split(X_te)

    print("[target] Encoded shapes (train / val / test):",
          X_tr_view.shape, X_va_view.shape, X_te_view.shape)

    feature_cols = list(X_tr_view.columns)
    return X_tr_view, X_va_view, X_te_view, feature_cols


# -------------------------
# TF-IDF + SVD VIEW
# -------------------------
def build_tfidf_view(ds_name, X_tr_raw, X_va_raw, X_te_raw, n_components=64):
    """
    Build TF-IDF bag-of-words over candidate cats.
    For each row, create a "document" of tokens like 'col_val'.
    Then apply TruncatedSVD → dense 64-dim representation.
    Concatenate with base numeric features.
    """
    cats, base_num_cols = _split_numeric_and_cats(ds_name, X_tr_raw, X_va_raw, X_te_raw)

    def _docs(df_raw):
        if not cats:
            return pd.Series([""] * len(df_raw), index=df_raw.index)
        df_tok = pd.DataFrame({
            c: df_raw[c].astype(str).fillna("Unknown")
            for c in cats if c in df_raw.columns
        })
        docs = df_tok.apply(
            lambda r: " ".join([f"{col}_{val}" for col, val in r.items()]),
            axis=1
        )
        return docs

    docs_tr = _docs(X_tr_raw)
    docs_va = _docs(X_va_raw)
    docs_te = _docs(X_te_raw)

    vect = TfidfVectorizer(min_df=1)
    tf_tr = vect.fit_transform(docs_tr)
    tf_va = vect.transform(docs_va)
    tf_te = vect.transform(docs_te)

    svd = TruncatedSVD(n_components=n_components, random_state=RSEED)
    Z_tr = svd.fit_transform(tf_tr)
    Z_va = svd.transform(tf_va)
    Z_te = svd.transform(tf_te)

    tf_cols = [f"tfidf_{i}" for i in range(Z_tr.shape[1])]

    def _combine(df_raw, Z):
        X_num = df_raw[base_num_cols].copy()
        Z_df = pd.DataFrame(Z, columns=tf_cols, index=df_raw.index)
        out = pd.concat([X_num, Z_df], axis=1)
        return out.astype(float)

    X_tr_view = _combine(X_tr_raw, Z_tr)
    X_va_view = _combine(X_va_raw, Z_va)
    X_te_view = _combine(X_te_raw, Z_te)

    print("[tfidf] Encoded shapes (train / val / test):",
          X_tr_view.shape, X_va_view.shape, X_te_view.shape)

    feature_cols = list(X_tr_view.columns)
    return X_tr_view, X_va_view, X_te_view, feature_cols


# -------------------------
# ENTITY EMBEDDING VIEW (small NN)
# -------------------------
class EntityNet4E(nn.Module):
    def __init__(self, num_numeric, cardinals, emb_dim=8, hidden_dim=32):
        super().__init__()
        self.emb_layers = nn.ModuleList([
            nn.Embedding(num_embeddings=c, embedding_dim=emb_dim)
            for c in cardinals
        ])
        self.fc1 = nn.Linear(num_numeric + emb_dim * len(cardinals), hidden_dim)
        self.act = nn.ReLU()
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x_cats, x_num):
        embs = [emb(x_cats[:, i]) for i, emb in enumerate(self.emb_layers)]
        if x_num is not None and x_num.shape[1] > 0:
            z = torch.cat(embs + [x_num], dim=1)
        else:
            z = torch.cat(embs, dim=1)
        h = self.act(self.fc1(z))
        logits = self.out(h).squeeze(1)
        return logits, h  # h is the learned representation


def build_entity_view(ds_name, X_tr_raw, X_va_raw, X_te_raw, y_tr,
                      emb_dim=8, hidden_dim=32, epochs=5, batch_size=256):
    """
    Train a small entity-embedding NN on candidate cats (+ numeric),
    then use the hidden layer h as the embedding features.
    """
    cats, base_num_cols = _split_numeric_and_cats(ds_name, X_tr_raw, X_va_raw, X_te_raw)

    # Categorical → codes with unknown bucket
    cat_code_tr, cat_code_va, cat_code_te = {}, {}, {}
    cardinals = []

    for c in cats:
        # TRAIN
        cat_tr = X_tr_raw[c].astype(str).fillna("Unknown").astype("category")
        codes_tr = np.asarray(cat_tr.cat.codes, dtype=np.int64)  # array, may be read-only but we don't modify
        categories = cat_tr.cat.categories
        unknown_idx = len(categories)
        num_embed = unknown_idx + 1
        cardinals.append(num_embed)
        cat_code_tr[c] = codes_tr

        # VAL
        cat_va = pd.Categorical(
            X_va_raw[c].astype(str).fillna("Unknown"),
            categories=categories
        )
        # make a *writable* copy
        codes_va = np.asarray(cat_va.codes, dtype=np.int64).copy()
        codes_va[codes_va < 0] = unknown_idx
        cat_code_va[c] = codes_va

        # TEST
        cat_te = pd.Categorical(
            X_te_raw[c].astype(str).fillna("Unknown"),
            categories=categories
        )
        # also make a writable copy
        codes_te = np.asarray(cat_te.codes, dtype=np.int64).copy()
        codes_te[codes_te < 0] = unknown_idx
        cat_code_te[c] = codes_te

    if not cardinals:
        raise ValueError("No candidate categorical columns found for entity view.")

    # Numeric features
    def _num_frame(df_raw):
        return df_raw[base_num_cols].astype(float).copy()

    X_tr_num = _num_frame(X_tr_raw)
    X_va_num = _num_frame(X_va_raw)
    X_te_num = _num_frame(X_te_raw)

    # Build categorical tensors [n_samples, n_cats]
    X_tr_cat = torch.tensor(
        np.vstack([cat_code_tr[c] for c in cats]).T,
        dtype=torch.long
    )
    X_va_cat = torch.tensor(
        np.vstack([cat_code_va[c] for c in cats]).T,
        dtype=torch.long
    )
    X_te_cat = torch.tensor(
        np.vstack([cat_code_te[c] for c in cats]).T,
        dtype=torch.long
    )

    # Numeric tensors
    X_tr_num_t = torch.tensor(X_tr_num.values, dtype=torch.float32)
    X_va_num_t = torch.tensor(X_va_num.values, dtype=torch.float32)
    X_te_num_t = torch.tensor(X_te_num.values, dtype=torch.float32)

    y_tr_t = torch.tensor(y_tr.values.astype(np.float32), dtype=torch.float32)

    ds = TensorDataset(X_tr_cat, X_tr_num_t, y_tr_t)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

    model = EntityNet4E(num_numeric=X_tr_num.shape[1], cardinals=cardinals,
                        emb_dim=emb_dim, hidden_dim=hidden_dim)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()

    model.train()
    for ep in range(epochs):
        ep_loss = 0.0
        for xb_cat, xb_num, yb in loader:
            opt.zero_grad()
            logits, _ = model(xb_cat, xb_num)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            ep_loss += loss.item() * len(yb)
        ep_loss /= len(loader.dataset)
        print(f"[entity] epoch {ep+1}/{epochs} - loss={ep_loss:.4f}")

    # Extract hidden representations
    model.eval()
    with torch.no_grad():
        _, H_tr = model(X_tr_cat, X_tr_num_t)
        _, H_va = model(X_va_cat, X_va_num_t)
        _, H_te = model(X_te_cat, X_te_num_t)

    H_cols = [f"entity_{i}" for i in range(H_tr.shape[1])]

    H_tr_df = pd.DataFrame(H_tr.numpy(), columns=H_cols, index=X_tr_raw.index)
    H_va_df = pd.DataFrame(H_va.numpy(), columns=H_cols, index=X_va_raw.index)
    H_te_df = pd.DataFrame(H_te.numpy(), columns=H_cols, index=X_te_raw.index)

    # Combine with numeric
    X_tr_view = pd.concat([X_tr_num, H_tr_df], axis=1)
    X_va_view = pd.concat([X_va_num, H_va_df], axis=1)
    X_te_view = pd.concat([X_te_num, H_te_df], axis=1)

    print("[entity] Encoded shapes (train / val / test):",
          X_tr_view.shape, X_va_view.shape, X_te_view.shape)

    feature_cols = list(X_tr_view.columns)
    return X_tr_view, X_va_view, X_te_view, feature_cols

# -------------------------
# Dispatcher
# -------------------------
def encode_view(ds_name, view, X_tr_raw, X_va_raw, X_te_raw, y_tr):
    """
    Unified entry point for building X_tr, X_va, X_te and feature_cols
    for a given view.
    """
    if view == "onehot":
        return build_onehot_view(ds_name, X_tr_raw, X_va_raw, X_te_raw)
    if view == "target":
        return build_target_view(ds_name, X_tr_raw, X_va_raw, X_te_raw, y_tr)
    if view == "tfidf":
        return build_tfidf_view(ds_name, X_tr_raw, X_va_raw, X_te_raw)
    if view == "entity":
        return build_entity_view(ds_name, X_tr_raw, X_va_raw, X_te_raw, y_tr)

    raise ValueError(f"Unsupported view: {view}")


In [ ]:
# ====================
# 04e helpers / utils
# ====================
import numpy as np
import pandas as pd

from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score,
    precision_score, recall_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

RUNLOG = []

def _timed(name):
    """Simple timing decorator that logs wall-clock time (same style as 4d)."""
    def deco(fn):
        def wrapper(*args, **kwargs):
            import time
            t0 = time.time()
            print(f"\n  [{name}] start")
            out = fn(*args, **kwargs)
            dt = time.time() - t0
            print(f" [{name}] done in {dt:0.1f}s")
            RUNLOG.append({"step": name, "seconds": dt})
            return out
        return wrapper
    return deco

def pick_target(df, candidates):
    """Pick the first existing column from candidates as the binary target."""
    for c in candidates:
        if c in df.columns:
            return c, df
    raise ValueError(
        f"None of {candidates} found in dataframe columns: {list(df.columns)}"
    )

def make_model(tag, X_train, y_train):
    """Instantiate and fit a classifier from CONFIG['models'] (same as 4d)."""
    cfg = CONFIG["models"][tag]
    t = cfg["type"]
    params = cfg["params"].copy()

    if t == "lr":
        model = LogisticRegression(**params)
    elif t == "linsvm":
        model = LinearSVC(**params)
    elif t == "rf":
        model = RandomForestClassifier(**params)
    elif t == "lgbm":
        model = LGBMClassifier(**params)
    else:
        raise ValueError(t)

    model.fit(X_train, y_train)
    return model

def _pred_proba_or_decision(model, X):
    """Get a 1D score for AUC / thresholding (same logic as 4d)."""
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        s = model.decision_function(X)
        if s.ndim == 1:
            return s
        return s[:, 1]
    return model.predict(X)

def score_suite(tag, X_train, y_train, X_test, y_test, feature_cols):
    """
    Train all models on X_train and evaluate on X_test.
    Returns a small DataFrame with AUC, F1, etc.
    """
    rows = []
    Xtr = X_train[feature_cols].values
    Xte = X_test[feature_cols].values

    for mtag in CONFIG["models"].keys():
        print(f"   → [{tag}] model={mtag}")
        model = make_model(mtag, Xtr, y_train.values)
        scores = _pred_proba_or_decision(model, Xte)

        # convert scores → predicted labels using 0.5 threshold for proba,
        # or 0-threshold for decision scores (same as 4d logic).
        if scores.min() >= 0 and scores.max() <= 1:
            y_hat = (scores >= 0.5).astype(int)
        else:
            y_hat = (scores >= 0).astype(int)

        auc  = roc_auc_score(y_test, scores)
        f1   = f1_score(y_test, y_hat)
        acc  = accuracy_score(y_test, y_hat)
        prec = precision_score(y_test, y_hat)
        rec  = recall_score(y_test, y_hat)

        rows.append({
            "Tag": tag,
            "Model": mtag,
            "AUC": auc,
            "F1": f1,
            "ACC": acc,
            "PREC": prec,
            "REC": rec
        })

    return pd.DataFrame(rows)

def _labels_ok(y):
    """Check that we have a proper binary label with both classes present."""
    vals = pd.Series(y).dropna().unique().tolist()
    vals = sorted(vals)
    return len(vals) == 2 and set(vals) == {0, 1}


In [ ]:
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.model_selection import train_test_split
import os

OUT_ROOT_4E = f"{PROJ}/outputs/synthetic_eval_4e"
os.makedirs(OUT_ROOT_4E, exist_ok=True)

@_timed("run_dataset_view_4e")
def run_dataset_view_4e(ds_name, view):
    ds_cfg  = DATASETS[ds_name]
    out_dir = f"{OUT_ROOT_4E}/{ds_name}_{view}"
    os.makedirs(out_dir, exist_ok=True)

    print(f"\n==============================")
    print(f" Dataset = {ds_name.upper()} | View = {view}")
    print(f"==============================")

    # ----- Load full cleaned dataset -----
    df_full = pd.read_csv(ds_cfg["path"], low_memory=False)
    print("Raw shape:", df_full.shape)

    # ----- Pick target -----
    target, df_full = pick_target(df_full, ds_cfg["target_candidates"])
    print(f"Using target column: {target}")

    y = df_full[target].astype(int)
    X_raw = df_full.drop(columns=[target])

    # ----- Train/val/test splits on RAW features -----
    X_tr_raw, X_tmp_raw, y_tr, y_tmp = train_test_split(
        X_raw, y,
        train_size=CONFIG["train_frac"],
        stratify=y,
        random_state=RSEED
    )
    X_va_raw, X_te_raw, y_va, y_te = train_test_split(
        X_tmp_raw, y_tmp,
        test_size=0.5,
        stratify=y_tmp,
        random_state=RSEED
    )

    print("Raw shapes (train / val / test):",
          X_tr_raw.shape, X_va_raw.shape, X_te_raw.shape)

    if not _labels_ok(y_tr):
        raise ValueError("Training labels are not proper binary {0,1}.")

    # ----- Encode according to view -----
    X_tr, X_va, X_te, num_cols = encode_view(
        ds_name, view,
        X_tr_raw, X_va_raw, X_te_raw,
        y_tr
    )
    print(f"Encoded shapes (train / val / test):",
          X_tr.shape, X_va.shape, X_te.shape)

    # ----- SDV metadata (all encoded features numerical, target categorical) -----
    @_timed(f"{ds_name}_{view}:build_metadata_4e")
    def build_metadata_full(Xtrain, ytrain):
        train_full = Xtrain.copy()
        train_full[target] = ytrain.values

        meta = SingleTableMetadata()
        meta.detect_from_dataframe(train_full)

        for c in num_cols:
            meta.update_column(c, sdtype="numerical")
        meta.update_column(target, sdtype="categorical")
        meta.validate()
        return meta, train_full

    meta_full, train_full = build_metadata_full(X_tr, y_tr)

    # ----- Fit TVAE ONLY (CTGAN disabled for 4E) -----
    ctgan = None
    tvae  = None

    # We intentionally skip CTGAN here because high-cardinality
    # views cause dimensional blow-ups and instability.
    use_ctgan = False  # hard-disable CTGAN in 4E

    # (Keep this around in case you ever want to re-enable CTGAN,
    #  but it will not be called while use_ctgan=False.)
    @_timed(f"{ds_name}_{view}:fit_CTGAN_4e")
    def fit_ctgan(train_full, meta):
        cfg = CONFIG["ctgan"]
        synth = CTGANSynthesizer(
            metadata=meta,
            epochs=cfg["epochs"],
            batch_size=cfg["batch_size"],
            pac=cfg["pac"],
            verbose=cfg["verbose"],
        )
        synth.fit(train_full)
        return synth


    @_timed(f"{ds_name}_{view}:fit_TVAE_4e")
    def fit_tvae(train_full, meta):
        cfg = CONFIG["tvae"]
        synth = TVAESynthesizer(
            metadata=meta,
            epochs=cfg["epochs"],
            batch_size=cfg["batch_size"],
            compress_dims=cfg["compress_dims"],
        )
        synth.fit(train_full)
        return synth

    if use_ctgan:
        ctgan = fit_ctgan(train_full, meta_full)
    else:
        print(f"Skipping CTGAN for view={view} (too many columns: {len(num_cols)}).")

    tvae = fit_tvae(train_full, meta_full)

    # ----- Sample synthetic data -----
    MAX_SYNTH_ROWS = 15000  # keep aligned with 4d

    def sample_synth(synth, n_rows, tag):
        print(f"\nSampling {n_rows} rows from {tag} ...")
        df_syn = synth.sample(num_rows=n_rows)
        if target not in df_syn.columns:
            raise ValueError(f"{tag}: target column '{target}' missing in synthetic sample.")
        df_syn = df_syn.dropna(subset=[target])
        df_syn[target] = df_syn[target].round().astype(int)
        return df_syn

    X_syn_ctgan = y_syn_ctgan = None
    X_syn_tvae  = y_syn_tvae  = None

    # --- CTGAN synthetic ---
    if ctgan is not None:
        try:
            n_ctgan = min(len(X_tr), MAX_SYNTH_ROWS)
            df_ctgan = sample_synth(
                ctgan,
                n_rows=n_ctgan,
                tag=f"{ds_name}_{view}_CTGAN_4e"
            )
            y_syn_ctgan = df_ctgan[target].astype(int)
            X_syn_ctgan = df_ctgan[num_cols].copy()
            X_syn_ctgan = X_syn_ctgan.replace([np.inf, -np.inf], np.nan)
            X_syn_ctgan = X_syn_ctgan.fillna(X_syn_ctgan.median(numeric_only=True))
        except Exception as e:
            print("  CTGAN sampling failed:", e)

    # --- TVAE synthetic ---
    try:
        n_tvae = min(len(X_tr), MAX_SYNTH_ROWS)
        df_tvae = sample_synth(
            tvae,
            n_rows=n_tvae,
            tag=f"{ds_name}_{view}_TVAE_4e"
        )
        y_syn_tvae = df_tvae[target].astype(int)
        X_syn_tvae = df_tvae[num_cols].copy()
        X_syn_tvae = X_syn_tvae.replace([np.inf, -np.inf], np.nan)
        X_syn_tvae = X_syn_tvae.fillna(X_syn_tvae.median(numeric_only=True))
    except Exception as e:
        print("  TVAE sampling failed:", e)

    # ----- TSTR / TRTS evaluation -----
    summaries = []

    if X_syn_ctgan is not None and _labels_ok(y_syn_ctgan):
        summaries.append(
            score_suite(
                tag="tstr_ctgan",
                X_train=X_syn_ctgan, y_train=y_syn_ctgan,
                X_test=X_te,        y_test=y_te,
                feature_cols=num_cols,
            )
        )
        summaries.append(
            score_suite(
                tag="trts_ctgan",
                X_train=X_tr,       y_train=y_tr,
                X_test=X_syn_ctgan, y_test=y_syn_ctgan,
                feature_cols=num_cols,
            )
        )
    else:
        print("  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).")

    if X_syn_tvae is not None and _labels_ok(y_syn_tvae):
        summaries.append(
            score_suite(
                tag="tstr_tvae",
                X_train=X_syn_tvae, y_train=y_syn_tvae,
                X_test=X_te,        y_test=y_te,
                feature_cols=num_cols,
            )
        )
        summaries.append(
            score_suite(
                tag="trts_tvae",
                X_train=X_tr,       y_train=y_tr,
                X_test=X_syn_tvae,  y_test=y_syn_tvae,
                feature_cols=num_cols,
            )
        )
    else:
        print("  Skipping TVAE TSTR/TRTS: synthetic labels not bi-class or sampling failed.")

    if len(summaries) == 0:
        print("  No valid TSTR/TRTS summaries for this dataset/view.")
        summary = pd.DataFrame(columns=["Tag","Model","AUC","F1","ACC","PREC","REC"])
    else:
        summary = pd.concat(summaries, axis=0, ignore_index=True)

    # Add metadata columns for aggregation
    summary.insert(0, "Dataset", ds_name.upper())
    summary.insert(1, "View", view)

    out_csv = os.path.join(out_dir, "summary_tstr_trts_4e.csv")
    summary.to_csv(out_csv, index=False)
    print("\n Saved summary →", out_csv)

    return summary


In [ ]:
from IPython.display import display

# ---------------------------------------
# Adult: views = onehot, target
# ---------------------------------------
adult_summaries_4e_fast = []

for view in ["onehot", "target"]:
    print("\n======================")
    print("Adult | view =", view)
    print("======================")
    try:
        summary = run_dataset_view_4e("adult", view)
        adult_summaries_4e_fast.append(summary)
    except Exception as e:
        print(f" Failed for adult, view={view}: {e}")

if adult_summaries_4e_fast:
    adult_fast_combined = pd.concat(adult_summaries_4e_fast, axis=0, ignore_index=True)
    display(adult_fast_combined.head())

    # Save / append to global 4E summary
    agg_path_4e = os.path.join(OUT_ROOT_4E, "summary_all_datasets_views_4e.csv")
    if os.path.exists(agg_path_4e):
        base_df = pd.read_csv(agg_path_4e)
        merged = pd.concat([base_df, adult_fast_combined], axis=0, ignore_index=True)
        merged.to_csv(agg_path_4e, index=False)
    else:
        adult_fast_combined.to_csv(agg_path_4e, index=False)

    print("\n Updated 4E combined summary:", agg_path_4e)



Adult | view = onehot

⏱️  [run_dataset_view_4e] start

 Dataset = ADULT | View = onehot
Raw shape: (48842, 14)
Using target column: income
Raw shapes (train / val / test): (24421, 13) (12210, 13) (12211, 13)
[onehot] Encoded shapes (train / val / test): (24421, 70) (12210, 70) (12211, 70)
Encoded shapes (train / val / test): (24421, 70) (12210, 70) (12211, 70)

⏱️  [adult_onehot:build_metadata_4e] start
✅ [adult_onehot:build_metadata_4e] done in 0.8s
Skipping CTGAN for view=onehot (too many columns: 70).

⏱️  [adult_onehot:fit_TVAE_4e] start


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [adult_onehot:fit_TVAE_4e] done in 414.5s

Sampling 15000 rows from adult_onehot_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
⚠️  Skipping TVAE TSTR/TRTS: synthetic labels not bi-class or sampling failed.
⚠️  No valid TSTR/TRTS summaries for this dataset/view.

🧾 Saved summary → /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/adult_onehot/summary_tstr_trts_4e.csv
✅ [run_dataset_view_4e] done in 418.2s

Adult | view = target

⏱️  [run_dataset_view_4e] start

 Dataset = ADULT | View = target
Raw shape: (48842, 14)
Using target column: income
Raw shapes (train / val / test): (24421, 13) (12210, 13) (12211, 13)
[target] Encoded shapes (train / val / test): (24421, 8) (12210, 8) (12211, 8)
Encoded shapes (train / val / test): (24421, 8) (12210, 8) (12211, 8)

⏱️  [adult_target:build_metadata_4e] start
✅ [adult_target:build_metadata_4e] done in 0.0s
Skipping CTGAN for view=target (too many columns: 8).

⏱️  [adult_target:fit_TVAE_4e] 

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [adult_target:fit_TVAE_4e] done in 104.6s

Sampling 15000 rows from adult_target_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 4595, number of negative: 10405
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1140
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.306333 -> initscore=-0.817318
[LightGBM] [Info] Start training from score -0.817318
   → [trts_tvae] model=LR
   → [trts_tvae] model=LinSVM
   → [trts_tvae] model=RF
   → [trts_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 5844, number of 

/tmp/ipython-input-2303222988.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  adult_fast_combined = pd.concat(adult_summaries_4e_fast, axis=0, ignore_index=True)


,Dataset,View,Tag,Model,AUC,F1,ACC,PREC,REC
0,ADULT,target,tstr_tvae,LR,0.790885,0.552108,0.756449,0.493007,0.627310
1,ADULT,target,tstr_tvae,LinSVM,0.785008,0.547915,0.762018,0.502282,0.602669
2,ADULT,target,tstr_tvae,RF,0.802425,0.445357,0.800917,0.668036,0.334018
3,ADULT,target,tstr_tvae,LGBM,0.820032,0.482530,0.808370,0.681875,0.373374
4,ADULT,target,trts_tvae,LR,0.959592,0.604149,0.811733,0.848759,0.468988



✅ Updated 4E combined summary: /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/summary_all_datasets_views_4e.csv


In [ ]:
from IPython.display import display

# ---------------------------------------
# Adult: views = tfidf, entity
# ---------------------------------------
adult_summaries_4e_heavy = []

for view in ["tfidf", "entity"]:
    print("\n======================")
    print("Adult | view =", view)
    print("======================")
    try:
        summary = run_dataset_view_4e("adult", view)
        adult_summaries_4e_heavy.append(summary)
    except Exception as e:
        print(f" Failed for adult, view={view}: {e}")

if adult_summaries_4e_heavy:
    adult_heavy_combined = pd.concat(adult_summaries_4e_heavy, axis=0, ignore_index=True)
    display(adult_heavy_combined.head())

    agg_path_4e = os.path.join(OUT_ROOT_4E, "summary_all_datasets_views_4e.csv")
    if os.path.exists(agg_path_4e):
        base_df = pd.read_csv(agg_path_4e)
        merged = pd.concat([base_df, adult_heavy_combined], axis=0, ignore_index=True)
        merged.to_csv(agg_path_4e, index=False)
    else:
        adult_heavy_combined.to_csv(agg_path_4e, index=False)

    print("\n Updated 4E combined summary:", agg_path_4e)



Adult | view = tfidf

⏱️  [run_dataset_view_4e] start

 Dataset = ADULT | View = tfidf
Raw shape: (48842, 14)
Using target column: income
Raw shapes (train / val / test): (24421, 13) (12210, 13) (12211, 13)
[tfidf] Encoded shapes (train / val / test): (24421, 69) (12210, 69) (12211, 69)
Encoded shapes (train / val / test): (24421, 69) (12210, 69) (12211, 69)

⏱️  [adult_tfidf:build_metadata_4e] start
✅ [adult_tfidf:build_metadata_4e] done in 0.3s
Skipping CTGAN for view=tfidf (too many columns: 69).

⏱️  [adult_tfidf:fit_TVAE_4e] start


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [adult_tfidf:fit_TVAE_4e] done in 651.8s

Sampling 15000 rows from adult_tfidf_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 684, number of negative: 14316
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15968
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 69
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.045600 -> initscore=-3.041175
[LightGBM] [Info] Start training from score -3.041175
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [adult_entity:fit_TVAE_4e] done in 344.2s

Sampling 15000 rows from adult_entity_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 1446, number of negative: 13554
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8119
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096400 -> initscore=-2.237881
[LightGBM] [Info] Start training from score -2.237881
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

,Dataset,View,Tag,Model,AUC,F1,ACC,PREC,REC
0,ADULT,tfidf,tstr_tvae,LR,0.751970,0.455500,0.787569,0.589034,0.371321
1,ADULT,tfidf,tstr_tvae,LinSVM,0.742730,0.429036,0.784211,0.584761,0.338809
2,ADULT,tfidf,tstr_tvae,RF,0.730958,0.133494,0.765048,0.568123,0.075633
3,ADULT,tfidf,tstr_tvae,LGBM,0.759166,0.142212,0.766850,0.594458,0.080767
4,ADULT,tfidf,trts_tvae,LR,0.942697,0.437015,0.937133,0.369324,0.535088



✅ Updated 4E combined summary: /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/summary_all_datasets_views_4e.csv


In [ ]:
from IPython.display import display

# ---------------------------------------
# PetFinder: views = onehot, target, tfidf, entity
# ---------------------------------------
pet_summaries_4e = []

for view in ["onehot", "target", "tfidf", "entity"]:
    print("\n======================")
    print("PetFinder | view =", view)
    print("======================")
    try:
        summary = run_dataset_view_4e("petfinder", view)
        pet_summaries_4e.append(summary)
    except Exception as e:
        print(f" Failed for petfinder, view={view}: {e}")

if pet_summaries_4e:
    pet_combined = pd.concat(pet_summaries_4e, axis=0, ignore_index=True)
    display(pet_combined.head())

    agg_path_4e = os.path.join(OUT_ROOT_4E, "summary_all_datasets_views_4e.csv")
    if os.path.exists(agg_path_4e):
        base_df = pd.read_csv(agg_path_4e)
        merged = pd.concat([base_df, pet_combined], axis=0, ignore_index=True)
        merged.to_csv(agg_path_4e, index=False)
    else:
        pet_combined.to_csv(agg_path_4e, index=False)

    print("\n Updated 4E combined summary:", agg_path_4e)



PetFinder | view = onehot

⏱️  [run_dataset_view_4e] start

 Dataset = PETFINDER | View = onehot
Raw shape: (11537, 14)
Using target column: AdoptionSpeed_bin
Raw shapes (train / val / test): (5768, 13) (2884, 13) (2885, 13)
[onehot] Encoded shapes (train / val / test): (5768, 145) (2884, 145) (2885, 145)
Encoded shapes (train / val / test): (5768, 145) (2884, 145) (2885, 145)

⏱️  [petfinder_onehot:build_metadata_4e] start
✅ [petfinder_onehot:build_metadata_4e] done in 0.2s
Skipping CTGAN for view=onehot (too many columns: 145).

⏱️  [petfinder_onehot:fit_TVAE_4e] start


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [petfinder_onehot:fit_TVAE_4e] done in 212.5s

Sampling 5768 rows from petfinder_onehot_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
⚠️  Skipping TVAE TSTR/TRTS: synthetic labels not bi-class or sampling failed.
⚠️  No valid TSTR/TRTS summaries for this dataset/view.

🧾 Saved summary → /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/petfinder_onehot/summary_tstr_trts_4e.csv
✅ [run_dataset_view_4e] done in 215.7s

PetFinder | view = target

⏱️  [run_dataset_view_4e] start

 Dataset = PETFINDER | View = target
Raw shape: (11537, 14)
Using target column: AdoptionSpeed_bin
Raw shapes (train / val / test): (5768, 13) (2884, 13) (2885, 13)
[target] Encoded shapes (train / val / test): (5768, 6) (2884, 6) (2885, 6)
Encoded shapes (train / val / test): (5768, 6) (2884, 6) (2885, 6)

⏱️  [petfinder_target:build_metadata_4e] start
✅ [petfinder_target:build_metadata_4e] done in 0.0s
Skipping CTGAN for view=target (too many columns: 6).

⏱️

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [petfinder_target:fit_TVAE_4e] done in 16.8s

Sampling 5768 rows from petfinder_target_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
⚠️  Skipping TVAE TSTR/TRTS: synthetic labels not bi-class or sampling failed.
⚠️  No valid TSTR/TRTS summaries for this dataset/view.

🧾 Saved summary → /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/petfinder_target/summary_tstr_trts_4e.csv
✅ [run_dataset_view_4e] done in 17.1s

PetFinder | view = tfidf

⏱️  [run_dataset_view_4e] start

 Dataset = PETFINDER | View = tfidf
Raw shape: (11537, 14)
Using target column: AdoptionSpeed_bin
Raw shapes (train / val / test): (5768, 13) (2884, 13) (2885, 13)
[tfidf] Encoded shapes (train / val / test): (5768, 67) (2884, 67) (2885, 67)
Encoded shapes (train / val / test): (5768, 67) (2884, 67) (2885, 67)

⏱️  [petfinder_tfidf:build_metadata_4e] start
✅ [petfinder_tfidf:build_metadata_4e] done in 0.1s
Skipping CTGAN for view=tfidf (too many columns: 67).

⏱️ 

...

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [petfinder_entity:fit_TVAE_4e] done in 91.1s

Sampling 5768 rows from petfinder_entity_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
⚠️  Skipping TVAE TSTR/TRTS: synthetic labels not bi-class or sampling failed.
⚠️  No valid TSTR/TRTS summaries for this dataset/view.

🧾 Saved summary → /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/petfinder_entity/summary_tstr_trts_4e.csv
✅ [run_dataset_view_4e] done in 92.2s


,Dataset,View,Tag,Model,AUC,F1,ACC,PREC,REC



✅ Updated 4E combined summary: /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/summary_all_datasets_views_4e.csv


/tmp/ipython-input-3532388028.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat([base_df, pet_combined], axis=0, ignore_index=True)


In [ ]:
from IPython.display import display

# ---------------------------------------
# Breast: views = target, tfidf, entity
# (onehot might be too wide; we can add it back if it passes the limit)
# ---------------------------------------
breast_summaries_4e = []

for view in ["target", "tfidf", "entity"]:
    print("\n======================")
    print("Breast | view =", view)
    print("======================")
    try:
        summary = run_dataset_view_4e("breast", view)
        breast_summaries_4e.append(summary)
    except Exception as e:
        print(f" Failed for breast, view={view}: {e}")

if breast_summaries_4e:
    breast_combined = pd.concat(breast_summaries_4e, axis=0, ignore_index=True)
    display(breast_combined.head())

    agg_path_4e = os.path.join(OUT_ROOT_4E, "summary_all_datasets_views_4e.csv")
    if os.path.exists(agg_path_4e):
        base_df = pd.read_csv(agg_path_4e)
        merged = pd.concat([base_df, breast_combined], axis=0, ignore_index=True)
        merged.to_csv(agg_path_4e, index=False)
    else:
        breast_combined.to_csv(agg_path_4e, index=False)

    print("\n Updated 4E combined summary:", agg_path_4e)



Breast | view = target

⏱️  [run_dataset_view_4e] start

 Dataset = BREAST | View = target
Raw shape: (59784, 126)
Using target column: OS5yr_bin
Raw shapes (train / val / test): (29892, 125) (14946, 125) (14946, 125)
[target] Encoded shapes (train / val / test): (29892, 71) (14946, 71) (14946, 71)
Encoded shapes (train / val / test): (29892, 71) (14946, 71) (14946, 71)

⏱️  [breast_target:build_metadata_4e] start
✅ [breast_target:build_metadata_4e] done in 0.3s
Skipping CTGAN for view=target (too many columns: 71).

⏱️  [breast_target:fit_TVAE_4e] start


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [breast_target:fit_TVAE_4e] done in 508.2s

Sampling 15000 rows from breast_target_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 4290, number of negative: 10710
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3010
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.286000 -> initscore=-0.914891
[LightGBM] [Info] Start training from score -0.914891
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [breast_tfidf:fit_TVAE_4e] done in 1435.0s

Sampling 15000 rows from breast_tfidf_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 524, number of negative: 14476
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18489
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 114
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.034933 -> initscore=-3.318756
[LightGBM] [Info] Start training from score -3.318756
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ [breast_entity:fit_TVAE_4e] done in 887.1s

Sampling 15000 rows from breast_entity_TVAE_4e ...
⚠️  Skipping CTGAN TSTR/TRTS for this view (no CTGAN model or bad labels).
   → [tstr_tvae] model=LR
   → [tstr_tvae] model=LinSVM
   → [tstr_tvae] model=RF
   → [tstr_tvae] model=LGBM
[LightGBM] [Info] Number of positive: 494, number of negative: 14506
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005754 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9865
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032933 -> initscore=-3.379782
[LightGBM] [Info] Start training from score -3.379782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

,Dataset,View,Tag,Model,AUC,F1,ACC,PREC,REC
0,BREAST,target,tstr_tvae,LR,0.716796,0.512618,0.793256,0.879329,0.361754
1,BREAST,target,tstr_tvae,LinSVM,0.781487,0.617262,0.825840,0.909052,0.467275
2,BREAST,target,tstr_tvae,RF,0.993235,0.954571,0.973170,0.971857,0.937890
3,BREAST,target,tstr_tvae,LGBM,0.995832,0.958715,0.975244,0.961074,0.956367
4,BREAST,target,trts_tvae,LR,0.995601,0.956860,0.975600,0.967811,0.946154



✅ Updated 4E combined summary: /content/drive/MyDrive/dissertation/outputs/synthetic_eval_4e/summary_all_datasets_views_4e.csv
